# <font color='blue'>Ensino Einstein</font>
# <font color='blue'>Ciência de Dados na área da Saúde</font>
## <font color='blue'>Machine Learning em Python</font>

In [0]:
from IPython.display import Image
Image(url = 'images/processo_DataScience.png')

In [0]:
import warnings
warnings.filterwarnings("ignore")

### Definição do Problema de Negócio

Criação de um modelo preditivo que seja capaz de prever a chance de uma pessoa ser internada durante o processo de Triagem.

Primeiramente apenas os Sinais Vitais, posteriormente os Sinais Vitais em conjunto com as demais variáveis já pré selecionadas.

Dataset: HIAE_SSVV_v01.csv, HIAE_SSVV_v02.csv & HIAE_Triagem_v03.csv

Este dataset descreve os registros médicos entre pacientes do HIAE após a passagem pela Triagem, onde agora cada registro já possui a informação se o paciente internou ou não, de acordo com pré-processamento já realizado anteriormente.

### Informações sobre os atributos:

1. ...

### Extraindo e Carregando os Dados

In [0]:
# Carregando arquivo .csv
import pandas as pd
file = 'data/HIAE_SSVV_v02.csv'
df_raw = pd.read_csv(file, sep = ";")

### Análise Exploratória de Dados

#### Estatística Descritiva

In [0]:
# Visualizando as primeiras 10 linhas
df_raw.head(10)

In [0]:
# Visualizando as dimensões
df_raw.shape

Preparando o dataset mantendo apenas as variáveis numéricas.

In [0]:
# lista as colunas para referência
df_raw.columns

In [0]:
# Colunas para deletar
col2del = ['T', 'PAd', 'PAs', 'FR', 'P', 'SpO2', 'T_str', 'PA_str', 'FR_str', 'P_str', 'SpO2_str']

# Deleta as colunas
df_raw.drop(col2del, axis = 1, inplace = True) # 'axis' = 0 (row) | axis = 1 (column)

In [0]:
# Tipo de dados de cada atributo
df_raw.dtypes

In [0]:
# Sumário estatístico
df_raw.describe()

Em problemas de classificação pode ser necessário balancear as classes. Aqui existe uma clara desproporção entre as classes 0 (paciente não interna) e 1 (paciente interna).

In [0]:
# Distribuição das classes
df_raw.groupby('Interna_num').size()

In [0]:
# Balanceamento (Down-sample Majority Class)

from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df_raw[df_raw.Interna_num == 0]
df_minority = df_raw[df_raw.Interna_num == 1]
 
# Upsample minority class
df_majority_downsampled = resample(df_majority, 
                                   replace = False,    # sample without replacement
                                   n_samples = 1955,   # to match minority class
                                   random_state = 123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.Interna_num.value_counts()

In [0]:
df_downsampled

In [0]:
dados = df_downsampled

A correlação é o relacionamento entre 2 variáveis. O método mais comum para calcular correlação é o método de Pearson, que assume uma distribuição normal dos dados. Correlação de -1 mostra uma correlação negativa, enquanto uma correlação de +1 mostra uma correlação positiva. Uma correlação igual a 0 mostra que não há relacionamento entre as variáveis.

(!!!) Alguns algoritmos como Regressão Linear e Regressão Logística podem apresentar problemas de performance se houver atributos altamente correlacionados (colineares)

In [0]:
# Correlação de Pearson
dados.corr(method = 'pearson')

#### Visualização com Matplotlib

In [0]:
import matplotlib.pyplot as plt
# Gráficos em janela separada
%matplotlib inline

In [0]:
# Matriz de Correlação (- nome das variáveis)
correlations = dados.corr()

# Plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin = -1, vmax = 1)
fig.colorbar(cax)
plt.show()

In [0]:
dados.columns

In [0]:
colunas = ['T_num', 'PA_num', 'FR_num', 'P_num', 'SpO2_num', 'Interna_num']

In [0]:
# Matriz de Correlação (+ nome das variáveis)
correlations = dados.corr()

# Plot
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin = -1, vmax = 1)
fig.colorbar(cax)
ticks = np.arange(0, 6, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(colunas)
ax.set_yticklabels(colunas)
plt.show()

Skew (ou simetria) se refere a distribuição dos dados que é assumida ser normal ou Gaussiana (bell curve). Muitos algoritmos de Machine Learning consideram que os dados possuem uma distribuição normal. O conhecendo sobre a simetria dos dados permite que você faça uma preparação adequada e entregue o que o algoritmo espera receber.

In [0]:
# Verificando o Skew de cada atributo
dados.skew()

Density Plots são outra forma de visualizar a distribuição dos dados para cada atributo, apresentando curvas através do topo dos bins de um histograma. Assim, a distribuição dos dados usando um Density Plot pode se tornar um pouco mais identificável.

In [0]:
# Density Plot
dados.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False)
plt.show()

### Preparando os Dados para Machine Learning

Muitos algoritmos esperam receber os dados em um formato específico. Você deve preparar os dados de acordo com uma estrutura que seja adequada ao algoritmo que você irá utilizar, o que pode requerer transformações diferentes nos dados. Ainda, é possível que em alguns casos, bons resultados sejam obtidos sem um trabalho de pré-processamento, mas é uma boa prática criar diferentes transformações dos dados para realizar testes em diferentes algoritmos de Machine Learning.

#### Normalização - Método 1

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

Uma das primeiras tarefas dentro do pré-processamento é colocar os dados em uma mesma escala. Muitos algoritmos de Machine Learning vão se beneficiar desse processamento e produzir resultados melhores. Esta etapa também é chamada de Normalização e significa colocar os dados em uma escala entre 0 e 1.

(!!!) Importantíssimo para o processo de otimização, assim como para algoritmos como Regressão e Redes Neurais e algoritmos que usam medidas de distância, como KNN.

O scikit-learn possui uma função para esse processamento, chamada MinMaxScaler().

In [0]:
# Transformando os dados para a mesma escala (entre 0 e 1)

# Import dos módulos
from sklearn.preprocessing import MinMaxScaler

array = dados.values

# Separando o array em componentes de input (X) e output (Y)
X = array[:, 0:5]
Y = array[:, 5]

# Gerando a nova escala (normalizando os dados)
scaler = MinMaxScaler(feature_range = (0, 1))
rescaledX = scaler.fit_transform(X)

# Sumarizando os dados transformados
print("Dados Originais: \n\n", dados.values)
print("\nDados Normalizados: \n\n", rescaledX[0:5, :])

In [0]:
# Visualização com Seaborn
import seaborn as sns
from scipy import stats

sns.distplot(rescaledX[:, 1], fit = stats.laplace, kde = False)

#### Normalização - Método 2

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

No scikit-learn, Normalização se refere a ajustar a escala de cada observação (linha) de modo que ela tenha comprimento igual a 1 (chamado vetor de comprimento 1 em álgebra linear).

(!!!) Importantíssimo quando temos Datasets esparsos (com muitos zeros) e atributos com escala muito variada, assim como para algoritmos de Redes Neurais ou que usam medida de distância, como KNN.

O scikit-learn possui uma função para esta etapa, chamada Normalizer().

In [0]:
# Normalizando os dados (comprimento igual a 1)
from sklearn.preprocessing import Normalizer

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:3]
Y = array[:, 3]

# Gerando os dados normalizados
scaler = Normalizer().fit(X)
normalizedX = scaler.transform(X)

# Sumarizando os dados transformados
print("Dados Originais: \n\n", dados.values)
print("\nDados Normalizados: \n\n", normalizedX[0:5, :])

In [0]:
sns.distplot(normalizedX[:, 1], fit = stats.laplace, kde = False)

#### Padronização

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

Padronização é a técnica para transformar os atributos com distribuição Gaussiana (normal) e diferentes médias e desvios padrão em uma distribuição Gaussiana com a média igual a 0 e desvio padrão igual a 1.

(!!!) Importantíssimo para algoritmos que esperam que os dados estejam com uma distribuição Gaussiana, como Regressão Linear, Regressão Logística e Linear Discriminant Analysis (LDA). Funciona bem quando os dados já estão na mesma escala.

O scikit-learn possui uma função para esta etapa, chamada StandardScaler().

In [0]:
# Padronizando os dados (0 para a média, 1 para o desvio padrão)

# Import dos módulos
from sklearn.preprocessing import StandardScaler

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5]
Y = array[:, 5]

# Gerando o novo padrão
scaler = StandardScaler().fit(X)
standardX = scaler.transform(X)

# Sumarizando os dados transformados
print("Dados Originais: \n\n", dados.values)
print("\nDados Padronizados: \n\n", standardX[0:5,:])

In [0]:
sns.distplot(standardX[:, 1], fit = stats.laplace, kde = False)


### Feature Selection

Os atributos presentes no seu Dataset e que você utiliza nos dados de treino terão grande influência na precisão e resultado do seu modelo preditivo.

(!!!) Importantíssimo notar que atributos irrelevantes geram um impacto negativo na performance, enquanto que atributos colineares podem afetar o grau de acurácia do modelo.

O scikit-learn possui funções que automatizam o trabalho de extração e seleção de variáveis.

A etapa de Feature Selection é onde selecionamos os atributos (variáveis) que serão melhores candidatas a variáveis preditoras. O Feature Selection nos ajuda a reduzir o overfitting (quando o algoritmo aprende demais), aumenta a acurácia do modelo e reduz o tempo de treinamento.

#### Eliminação Recursiva de Atributos (Recursive Feature Elimination - RFE)

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

Técnica para seleção de atributos que recursivamente remove os atributos e constrói o modelo com os atributos remanescentes. Utiliza a acurácia do modelo para identificar os atributos que mais contribuem para prever a variável alvo. Em inglês esta técnia é chamada Recursive Feature Elimination (RFE).

O exemplo abaixo utiliza a técnica de eliminação recursiva de atributos com um algoritmo de Regressão Logística para selecionar as 3 melhores variáveis preditoras. O RFE selecionou as variáveis PA_num, FR_num e SpO2_num, as quais marcadas como True em "Atributos Selecionados" e com valor 1 em "Ranking dos Atributos".

In [0]:
# Eliminação Recursiva de Variáveis

# Import dos módulos
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5]
Y = array[:, 5]

# Criação do modelo
modelo = LogisticRegression()

# RFE
rfe = RFE(modelo, 3)
fit = rfe.fit(X, Y)

# Print dos resultados
print("Variáveis Preditoras:", dados.columns[0:8])
print("Variáveis Selecionadas: %s" % fit.support_)
print("Ranking dos Atributos: %s" % fit.ranking_)
print("Número de Melhores Atributos: %d" % fit.n_features_)

#### Método Ensemble para Seleção de Variáveis (ExtraTreesClassifier)

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

Bagged Decision Trees, como o algoritmo RandomForest (esses são chamados de Métodos Ensemble), podem ser usados para estimar a importância de cada atributo, retornando um score para cada um dos mesmos.

In [0]:
# Importância do Atributo com o Extra Trees Classifier

# Import dos Módulos
from sklearn.ensemble import ExtraTreesClassifier

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5]
Y = array[:, 5]

# Criação do Modelo - Feature Selection
modelo = ExtraTreesClassifier()
modelo.fit(X, Y)

# Print dos Resultados
print(dados.columns[0:5])
print(modelo.feature_importances_)

In [0]:
# Colunas para deletar
col2del = ['T_num', 'P_num']

# Deleta as colunas
df_raw.drop(col2del, axis = 1, inplace = True) # 'axis' = 0 (row) | axis = 1 (column)

In [0]:
dados = df_raw
dados

### Amostragem - Resampling

A avaliação do modelo é uma estimativa de quão bem o algoritmo será capaz de prever em novos dados. Para tanto, devemos fazer previsões em dados que você já conhece o resultado, mas que o modelo não. Existem diversas técnicas para isso e estudaremos duas aqui: Conjunto de dados de treino e de teste e Cross Validation.

#### Dados de Treino e de Teste

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Método mais utilizado para avaliar a performance de um algoritmo de Machine Learning. Divide-se os dados originais em dados de treino e de teste. Treina-se o algoritmo nos dados de treino e então se faze as previsões nos dados de teste e avalia-se o resultado. A divisão dos dados depende do seu Dataset, mas proporções entre 70/30 (treino/teste) e 65/35 (treino/teste) são muito frequentes.

Este método é bem veloz e ideal para conjuntos de dados muito grandes. O ponto negativo é a possibilidade de alta variância.

In [0]:
# Avaliação usando dados de treino e de teste

# Import dos módulos
from sklearn.model_selection import train_test_split

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # **
X = rescaledX     # ***
# X = normalizedX   # *
# X = standardX     # **
Y = array[:, 5]

# Definindo o tamanho das amostras
teste_size = 0.33

# Garante que os resultados podem ser reproduzidos
# Isso é importante para comparar a acurácia com outros algoritmos de Machine Learning.
seed = 7

# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Import dos módulos
from sklearn.linear_model import LogisticRegression

# Criação do modelo
modelo = LogisticRegression()

# Treinamento do modelo
modelo.fit(X_treino, Y_treino)

# Score do modelo nos dados de teste
result = modelo.score(X_teste, Y_teste)
print("Acurácia nos Dados de Teste: %.3f%%" % (result * 100.0))

#### Cross Validation

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Cross Validation é uma técnica que pode ser utilizada para avaliar a performance de um modelo com menos variância que a técnica de dividir os dados em treino/teste. Com esta técnica dividimos os dados em partes chamadas k-folds. Cada parte é chamada fold e o algoritmo é treinado em k-1 folds, isso é, cada fold é usado no treinamento de forma repetida, um fold por vez. Após executar o processo em k-1 folds, sumariza-se a performance em cada fold usando a média e o desvio padrão. O resultado é normalmente mais confiável e oferece maior acurácia ao modelo. A chave deste processo está em definir o correto valor de k, de modo que o número de folds represente adequadamente o número de repetições necessárias.

In [0]:
from IPython.display import Image
Image(url = 'images/cross-validation.jpg')

In [0]:
# Avaliação usando Cross Validation

# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para os folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Import dos módulos
from sklearn.linear_model import LogisticRegression

# Criando o modelo
modelo = LogisticRegression()
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Usamos a média e o desvio padrão
print("Acurácia Final: %.3f%%" % (resultado.mean() * 100.0))

### Avaliando a Performance

As métricas que você escolhe para avaliar a performance do modelo vão influenciar a forma como a performance é medida e comparada com modelos criados com outros algoritmos.

Vamos utilizar o mesmo algoritmo, mas com métricas diferentes e assim comparar os resultados. A função cross_validation.cross_val_score() será usada para avaliar a performance.

#### Métricas para Algoritmos de Classificação

https://scikit-learn.org/stable/modules/model_evaluation.html

In [0]:
# Acurácia
# Número de previsões corretas. É útil apenas quando existe um balanceamento entre as classes.

# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Import dos módulos
from sklearn.linear_model import LogisticRegression

# Criando o modelo
modelo = LogisticRegression()

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold, scoring = 'accuracy')

# Print dos resultados
print("Acurácia: %.3f" % (resultado.mean() * 100))

In [0]:
# Curva ROC 
# A Curva ROC permite analisar a métrica AUC (Area Under the Curve).
# Essa é uma métrica de performance para classificação binária, em que podemos definir as classes em positiavs e negativas.
# Problemas de classificação binária são um trade-off sentre Sensitivity e Specifity.
# Sensitivity é a taxa de verdadeiros positivos (TP). Ese é o número de instâncias positivas da primeira classe que foram previstas corretamente.
# Specifity é a taxa de verdadeiros negativos (TN). Esse é o número de instâncias da segunda classe que foram previstas corretamente.
# Valores acima de 0.5 indicam uma boa taxa de previsão.

# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # ***
# X = rescaledX     # **
X = normalizedX   # ****
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Import dos módulos
from sklearn.linear_model import LogisticRegression

# Criando o modelo
model = LogisticRegression()

# Cross Validation
resultado = cross_val_score(model, X, Y, cv = kfold, scoring = 'roc_auc')

# Print do resultado
print("AUC: %.3f" % (resultado.mean() * 100))

In [0]:
import matplotlib.patches as patches
from sklearn.metrics import roc_curve,auc
from scipy import interp

# plot arrows
fig1 = plt.figure(figsize=[12,12])
ax1 = fig1.add_subplot(111,aspect = 'equal')
ax1.add_patch(
    patches.Arrow(0.45,0.5,-0.25,0.25,width=0.3,color='green',alpha = 0.5)
    )
ax1.add_patch(
    patches.Arrow(0.5,0.45,0.25,-0.25,width=0.3,color='red',alpha = 0.5)
    )

tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
i = 1
for train,test in kfold.split(X,Y):
    prediction = model.fit(pd.DataFrame(X).iloc[train],pd.DataFrame(Y).iloc[train]).predict_proba(pd.DataFrame(X).iloc[test])
    fpr, tpr, t = roc_curve(Y[test], prediction[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1

plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

In [0]:
# Confusion Matrix
# Permite verificar a acurácia em um formato de tabela

# Import dos módulos
from sklearn.model_selection import train_test_split

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5]
# X = rescaledX
X = normalizedX
# X = standardX
Y = array[:, 5]

# Definindo o tamanho do conjunto de dados
teste_size = 0.33
seed = 7

# Dividindo os dados em treino e teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Import dos módulos
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Criando o modelo
model = LogisticRegression()
model.fit(X_treino, Y_treino)

# Fazendo as previsões e construindo a Confusion Matrix
previsoes = model.predict(X_teste)
matrix = confusion_matrix(Y_teste, previsoes)

# Imprimindo a Confusion Matrix
print(matrix)

In [0]:
# Relatório de Classificação

# Import dos módulos
from sklearn.model_selection import train_test_split

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5]
# X = rescaledX
X = normalizedX
# X = standardX
Y = array[:, 5]

# Definindo o tamanho do conjunto de dados
teste_size = 0.33
seed = 7

# Dividindo os dados em treino e teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Import dos módulos
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Criando o modelo
modelo = LogisticRegression()
modelo.fit(X_treino, Y_treino)

# Fazendo as previsões e construindo o relatório
previsoes = model.predict(X_teste)
report = classification_report(Y_teste, previsoes)

# Imprimindo o relatório
print(report)

## Algoritmos de Classificação

Não há como como saber qual algoritmo vai funcionar melhor na construção de um modelo antes de o testarmos com um Dataset. O ideal é testar alguns algoritmos e então escolher o que fornece melhor nível de precisão (nesse caso, já que estamos com nossas classes balanceadas e estamos definindo essa estratégia). Vamos testar um conjunto de algoritmos de classificação nas mesmas condições e validarmos nossas conclusões.

### Regressão Logística

Algoritmo Linear. O algoritmo de Regressão Logística assume que seus dados estão em uma Distribuição Normal para valores numéricos que podem ser modelados com classificação binária.

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = LogisticRegression()

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### Linear Discriminant Analysis (LDA)

Algoritmo Linear. Técnica estatística para classificação binária. Também assume que os dados estão em Distribuição Normal.

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5] # **
# X = rescaledX     # **
# X = normalizedX   # *
# X = standardX     # **
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = LinearDiscriminantAnalysis()

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### KNN (K-Nearest Neighbors)

Algoritmo Não-Linear que utiliza uma métrica de distância para encontrar o valor de K mais adequado as instâncias do Dataset de treino.

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5] # *
# X = rescaledX     # *
# X = normalizedX   # *
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
random_state = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = random_state)

# Criando o modelo
modelo = KNeighborsClassifier()

# Cross Validation
results = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### Naive Bayes

Algoritmo Não-Linear. Calcula a probabilidade de cada classe e a probabilidade condicional de cada classe dado uma variável de entrada. As probabilidades são então estimadas para os novos dados e multiplicadas, assumindo que são independentes (suposição simples ou Naive). Assume dados em distirbuição Gaussiana (Normal)

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = GaussianNB()

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### CART (Classification and Regression Trees)

Algoritmo Não-Linear. O algoritmo CART constrói uma árvore binária a partir do Dataset de treino. Cada atributo e cada valor de cada atributo são avaliados com o objetivo de reduzir a função de custo (Cost Function).

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = DecisionTreeClassifier()

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

## Seleção do Modelo Preditivo

Veremos que os algoritmos Naive Bayes e CART apresentaram o melhor nível de precisão.

In [0]:
# Import dos módulos
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5]
# X = rescaledX
X = normalizedX
# X = standardX
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Preparando a lista de modelos
modelos = []
modelos.append(('LR', LogisticRegression()))
modelos.append(('LDA', LinearDiscriminantAnalysis()))
modelos.append(('NB', GaussianNB()))
modelos.append(('KNN', KNeighborsClassifier()))
modelos.append(('CART', DecisionTreeClassifier()))

# Avaliando cada modelo em um loop
resultados = []
nomes = []

for nome, modelo in modelos:
    kfold = KFold(n_splits = num_folds, random_state = seed)
    cv_results = cross_val_score(modelo, X, Y, cv = kfold, scoring = 'accuracy')
    resultados.append(cv_results)
    nomes.append(nome)
    msg = "%s: %f (%f)" % (nome, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot para comparar os algoritmos
fig = plt.figure()
fig.suptitle('Comparação de Algoritmos de Classificação')
ax = fig.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(nomes)
plt.show()

### Otimização do Modelo - Ajuste de Hyperparâmetros

Todos os algoritmos de Machine Learning são parametrizados, o que significa que você pode ajustar a performance do seu modelo preditivo através do "tuning" (ajuste fino) dos parâmetros. 

(!!!) É importantíssimo encontrar a melhor combinação entre os parâmetros em cada algoritmo de Machine Learning, e esse processo também é chamado de Otimização de Hyperparâmetros.

O scikit-learn oferece dois métodos para otimização automática dos parâmetros: Grid Search Parameter Tuning e Random Search Parameter Tuning. 

#### Grid Search Parameter Tuning

Este método realiza combinações entre todos os parâmetros do algoritmo, criando um grid. Vamos experimentar este método utilizando o algoritmo de Regressão Logística. 

In [0]:
# Import dos módulos
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores que serão testados
valores_grid = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}

# Criando o modelo
# modelo = LogisticRegression()
modelo = LogisticRegression()

# Criando o grid
grid = GridSearchCV(estimator = modelo, param_grid = valores_grid)
grid.fit(X, Y)

# Print do resultado
print("Acurácia: %.3f" % (grid.best_score_ * 100))
print("Melhores Parâmetros do Modelo:\n", grid.best_estimator_)

#### Random Search Parameter Tuning

Este método gera amostras dos parâmetros dos algoritmos a partir de uma distribuição randômica uniforme para um número fixo de iterações. Um modelo é construído e testado para cada combinação de parâmetros.

In [0]:
# Import dos módulos
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]

# Definindo os valores que serão testados
seed = 7
iterations = 14

# Definindo os valores que serão testados
valores_grid = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}

# Criando o modelo
modelo = LogisticRegression()

# Criando o grid
rsearch = RandomizedSearchCV(estimator = modelo, 
                             param_distributions = valores_grid, 
                             n_iter = iterations, 
                             random_state = seed)
rsearch.fit(X, Y)

# Print dos resultados
print("Acurácia: %.3f" % (rsearch.best_score_ * 100))
print("Melhores Parâmetros do Modelo:\n", rsearch.best_estimator_)

## Otimizando Performance com Métodos Ensemble

Métodos Ensemble permitem aumentar consideravelmente o nível de precisão nas suas previsões. Vamos criar alguns dos Métodos Ensemble mais poderosos em Python, dos quais 3 principais para combinar previsões a partir de diferentes modelos:

Bagging - Para construção de múltiplos modelos (normalmente do mesmo tipo) a partir de diferentes subsets no dataset de treino.

Boosting - Para construção de múltiplos modelos (normalmente do mesmo tipo), onde cada modelo aprende a corrigir os erros gerados pelo modelo anterior, dentro da sequência de modelos criados.

Voting - Para construção de múltiplos modelos (normalmente de tipos diferentes) e estatísticas simples (como a média) são usadas para combinar as previsões.

### Bagged Decision Trees

Funciona bem quando existe alta variância nos dados

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5] # **
# X = rescaledX     # **
# X = normalizedX   # *
# X = standardX     # **
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Cria o modelo unitário (classificador fraco)
cart = DecisionTreeClassifier()

# Definindo o número de trees
num_trees = 100

# Criando o modelo bagging
modelo = BaggingClassifier(base_estimator = cart, n_estimators = num_trees, random_state = seed)

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### Random Forest

Random Forest é uma extensão do Baggig Decision Tree. Amostras do Dataset de treino são usadas com reposição, mas as árvores são criadas de uma forma que reduz a correlação entre classificadores individuais (Random Forest é um conjunto de árvores de decisão).

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5] # **
# X = rescaledX     # **
# X = normalizedX   # *
# X = standardX     # **
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Definindo o número de trees
num_trees = 100
max_features = 5

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = RandomForestClassifier(n_estimators = num_trees, max_features = max_features, random_state = seed)

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### AdaBoost

Algoritmos baseados em Boosting Ensemble criam uma sequência de modelos que tentam corrigir os erros dos modelos anteriores dentro da sequência. Uma vez criados, os modelos fazem previsões que podem receber um peso de acordo com sua acurácia e os resultados são combinados para criar uma previsão única final. 

O AdaBoost atribui pesos às instâncias no Dataset, definindo quão fácil ou difícil elas são para o processo de classificação, permitindo que o algoritmo tenha mais ou menos atenção às instâncias durante o processo de construção dos modelos.

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # **
# X = rescaledX     # **
# X = normalizedX   # *
X = standardX     # **
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Definindo o número de trees
num_trees = 30

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = AdaBoostClassifier(n_estimators = num_trees, random_state = seed)

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### Gradient Boosting

Também chamado Stochastic Gradient Boosting, é um dos métodos Ensemble mais sofisticados.

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

array = dados.values

# Separando o array em componentes de input e output
# X = array[:, 0:5] # *
# X = rescaledX     # *
X = normalizedX   # **
# X = standardX     # *
Y = array[:, 5]


# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Definindo o número de trees
num_trees = 100

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando o modelo
modelo = GradientBoostingClassifier(n_estimators = num_trees, random_state = seed)

# Cross Validation
resultado = cross_val_score(modelo, X, Y, cv = kfold)

# Print do resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))

### Voting Ensemble

Este é um dos métodos Ensemble mais simples. Este método cria dois ou mais modelos separados a partir do Dataset de treino. O Classificador Voting então utiliza a média das previsões de cada sub-modelo para fazer as previsões em novos conjuntos de dados. As previsões de cada sub-modelo podem receber pesos, através de parâmetros definidos manualmente ou através de heurística.

In [0]:
# Import dos módulos
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

array = dados.values

# Separando o array em componentes de input e output
X = array[:, 0:5] # ***
# X = rescaledX   # *
# X = normalizedX # ***
# X = standardX   # **
Y = array[:, 5]

# Definindo os valores para o número de folds
num_folds = 10
seed = 7

# Separando os dados em folds
kfold = KFold(num_folds, True, random_state = seed)

# Criando os modelos
estimators = []

modelo1 = LogisticRegression()
estimators.append(('logistic', modelo1))

modelo2 = DecisionTreeClassifier()
estimators.append(('cart', modelo2))

modelo3 = SVC()
estimators.append(('svm', modelo3))

# Criando o modelo ensemble
ensemble = VotingClassifier(estimators)

# Cross Validation
resultado = cross_val_score(ensemble, X, Y, cv = kfold)

# Resultado
print("Acurácia: %.3f" % (resultado.mean() * 100))